In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymatgen as mg
from matminer.featurizers.conversions import StrToComposition
from matminer.featurizers.composition import ElementProperty
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
import time
from collections import defaultdict

# Load list of ABC compositions from the ICSD, Springer Materials, and ASM

In [2]:
use_kfold = True
n_trials = 10
trees = 100
stability_threshold = 0

training_data_filename = 'training_data.csv'
original_data = pd.read_csv(f'../released_Data Files/{training_data_filename}')

output_filename = f'ML_predictions_{training_data_filename}'

# Isolate category I and II compounds

In [3]:
I_II = original_data['Type'] == 'I_II'
III_IV = original_data['Type'] == 'III_IV'

I_II_on_hull = []

for i, row in original_data[I_II].iterrows():
    e1 = row['EP 1 Ehull (eV/atom)'] # DFT stability of first HH configuration
    e2 = row['EP 2 Ehull (eV/atom)']
    e3 = row['EP 3 Ehull (eV/atom)']
    
    stability = min([e1,e2,e3])
    
    if stability <= stability_threshold:
        I_II_on_hull += [True]
    else:
        I_II_on_hull += [False]
    
I_II_on_hull = np.array(I_II_on_hull)

# Isolate category III and IV compounds

In [4]:
III_IV_on_hull = []

for i, row in original_data[III_IV].iterrows():
    e1 = row['EP 1 Ehull (eV/atom)']
    e2 = row['EP 2 Ehull (eV/atom)']
    e3 = row['EP 3 Ehull (eV/atom)']
    
    stability = min([e1,e2,e3])
    
    if stability <= stability_threshold:
        III_IV_on_hull += [True]
    else:
        III_IV_on_hull += [False]
III_IV_on_hull = np.array(III_IV_on_hull)

# Sort data into the four categories

In [5]:
I = original_data[I_II][I_II_on_hull]
II = original_data[I_II][~I_II_on_hull]
III = original_data[III_IV][III_IV_on_hull]
IV = original_data[III_IV][~III_IV_on_hull]

# Featurize data

### First define a quick helper function

In [6]:
# https://www.calculatoratoz.com/en/percent-ionic-character-calculator/Calc-900#:~:text=Percent%20ionic%20character%3F-,Percent%20ionic%20character%20is%20a%20measure%20of%20compound%20or%20molecule%27s,element%20B)%5E2)).

def cubical_octahedral_differences_and_elementals(t1,o1,o2):
    
    t1_o1o2 = np.subtract(t1,np.mean((o1,o2),axis=0))
    
    return t1_o1o2

In [7]:
all_features = []

for data in [I,II,III,IV]:

    formulas = pd.DataFrame(data['Composition'])

    # 1. Featurize each individual element in the Half Heusler composition (in increasing order of electropositivity - EP))
    
    ep_feat = ElementProperty.from_preset(preset_name="magpie")
    e_feat = ElementProperty.from_preset(preset_name="magpie")
    ep_1_comps = StrToComposition().featurize_dataframe(pd.DataFrame(data['EP 1 element']), 'EP 1 element')
    ep_1_features = e_feat.featurize_dataframe(ep_1_comps, col_id="composition")
    ep_2_comps = StrToComposition().featurize_dataframe(pd.DataFrame(data['EP 2 element']), 'EP 2 element')
    ep_2_features = e_feat.featurize_dataframe(ep_2_comps, col_id="composition").to_numpy()[:,2:]
    ep_3_comps = StrToComposition().featurize_dataframe(pd.DataFrame(data['EP 3 element']), 'EP 3 element')
    ep_3_features = e_feat.featurize_dataframe(ep_3_comps, col_id="composition").to_numpy()[:,2:]
    all_ep_features = np.concatenate([ep_1_features.to_numpy()[:,2:].reshape((len(ep_1_features),len(ep_2_features[0]),1)),\
                                    ep_2_features.reshape((len(ep_1_features),len(ep_2_features[0]),1)),\
                                    ep_3_features.reshape((len(ep_1_features),len(ep_2_features[0]),1))],axis=2)
    
    # 2. identify cubical and octahedral elements, get their respective stabilities
    
    cubical_indices = []
    octahedral_indices = []
    stabilities = []
    
    for i, row in data.iterrows():
        all_stabilities = row[['EP 1 Ehull (eV/atom)', 'EP 2 Ehull (eV/atom)','EP 3 Ehull (eV/atom)']].values
        stability_index = np.argmin(all_stabilities)
        cubical_indices += [stability_index]
        
        stabilities += [all_stabilities[stability_index]]
        all_indices = np.array([0,1,2])
        octahedral_indices += [all_indices[np.isin(all_indices,[stability_index],invert=True)]]

    cubicals = []
    octahedrals_1 = []
    octahedrals_2 = []

    for i in range(len(all_ep_features)):
        cubicals += [all_ep_features[i][:,cubical_indices[i]]]
        octahedrals_1 += [all_ep_features[i][:,octahedral_indices[i][0]]]
        octahedrals_2 += [all_ep_features[i][:,octahedral_indices[i][1]]]
    
    cubicals = np.array(cubicals)
    octahedrals_1 = np.array(octahedrals_1)
    octahedrals_2 = np.array(octahedrals_2)
    
    octahedral_avg = np.mean([octahedrals_1,octahedrals_2],axis=0)
    stabilities = np.array(stabilities)
    
    # 3. featurize the entire Half Heusler formula
    
    formulas_comps = StrToComposition().featurize_dataframe(formulas, 'Composition')
    formula_features = ep_feat.featurize_dataframe(formulas_comps, col_id="composition")
    
    # 4. calculate cubical-octahedral features
    
    cub_oct_diff = cubical_octahedral_differences_and_elementals(cubicals,octahedrals_1,octahedrals_2)
    
    # 5. combine to get all features
    combined_features = np.concatenate([np.reshape(stabilities,(len(stabilities),1)),formula_features.to_numpy()[:,2:],cub_oct_diff,cubicals,octahedral_avg],axis=1)
    all_features += [combined_features]

cubical_octahedral_feature_names = []
cubical_feature_names = []
octahedral_feature_names = []

# rename some feature names
for x in ep_1_features.columns[2:]:
    cubical_octahedral_feature_names += ['cub_oct_'+x]
    cubical_feature_names += ['cub_'+x]
    octahedral_feature_names += ['oct_'+x]

# get final list of feature names
training_feature_names = np.concatenate([['Stability'],formula_features.columns[2:],cubical_octahedral_feature_names,cubical_feature_names,octahedral_feature_names])

ElementProperty: 100%|██████████| 2200/2200 [00:21<00:00, 100.64it/s]


# Define function to get the electron count of a composition

In [8]:
def electronCount(chemFormula):
    # takes in a string composition
    
    chemFormula = mg.core.composition.Composition(chemFormula)
    
    symbolsOnly = list(chemFormula.keys())
    stoichOnly = list(chemFormula.values())
    
    electronCount = 0
    
    for i in range(len(symbolsOnly)):
        a = mg.core.Element(symbolsOnly[i])
        valence = a.full_electronic_structure[-2:]
        
        if a.is_rare_earth_metal: 
            electrons = 3
        else:
            electrons = int(valence[0][2]) + int(valence[1][2])
            
        electronCount += electrons * stoichOnly[i]

    electronCount /= max(stoichOnly)

    return electronCount

 # Define function to run ML

In [9]:
def run_ML(n_undersample,trial_number,n_trees,aggregated_feature_importances,training_feature_names):
    
    data_list = [I,II,III,IV]
    all_compositions = np.concatenate([data['Composition'].to_numpy() for data in data_list])
    training_labels = np.concatenate([[0]*len(I),[0]*len(II),[1]*len(III),[1]*len(IV)]) # 0 = synthesizable, 1 = unsynthesizable  
    
    training_features = np.concatenate(all_features)
    
    # find NaNs

    nan_counter_examples = 0

    nan_indices = [0]*training_features.shape[1]
    for examples in training_features:
        nan_counter = 0
        for i in range(len(examples)):
            if np.isnan(examples[i]): 
                nan_indices[i] += 1
                nan_counter +=1
        if nan_counter > 0: nan_counter_examples += 1


    # remove NaNs and impute the rest

    indices_to_remove = []
    for i in range(len(nan_indices)):
        if nan_indices[i] >= 10: #if more than 10 NaNs in the feature, remove the feature
            indices_to_remove += [i]
            print(f'Removing NaN feature -> {training_feature_names[i]}.')

    training_features = np.delete(training_features, indices_to_remove, axis=1)
    training_feature_names = np.delete(training_feature_names, indices_to_remove)
    
    
    # impute features with fewer than 10 NaNs

    imputer_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    imputer_mean.fit(training_features)
    training_features = imputer_mean.transform(training_features)    
    
    # undersample category IV data points and keep the non-training data to predict on later

    remove_IV = np.random.choice(range(len(I)+len(II)+len(III),len(I)+len(II)+len(III)+len(IV)),size=(len(IV)-n_undersample),replace=False)
    remove_IV = np.concatenate((remove_IV,range(len(I)+len(II)+len(III)+len(IV),len(training_features))))

    remove_IV = remove_IV.astype(dtype=int)
    
    keep_IV = list(set(range(len(training_features))).difference(set(remove_IV)))
    
    remaining_features = training_features[remove_IV]
    predicted_comps = all_compositions[remove_IV]

    training_features = training_features[keep_IV]
    training_labels = training_labels[keep_IV]
    training_compositions = all_compositions[keep_IV]

    
    # run ML

    model = RandomForestClassifier(n_estimators = n_trees,n_jobs=-1)
 
    splits = int(len(training_labels)/10) # leave ten out
    #splits = int(len(training_labels)) # leave one out
    kf = KFold(n_splits = splits,shuffle=True) 

    training_predicted = []
    training_predicted_comps = []


    # important to convert to numpy array
    training_features = np.array(training_features)
    training_labels = np.array(training_labels)
    training_compositions = np.array(training_compositions)
    
    for train_index, test_index in kf.split(training_features,training_labels):
        X_train, X_test = training_features[train_index], training_features[test_index]
        y_train, y_test = training_labels[train_index], training_labels[test_index]

        model.fit(X_train,y_train)
        y_predicted = model.predict_proba(X_test)

        training_predicted += [y_predicted]
        training_predicted_comps += [training_compositions[test_index]]

    train_predicted , train_predicted_comps = np.concatenate(training_predicted,axis=0),np.concatenate(training_predicted_comps)
    
    # still need to predict on remaining data points that were thrown out from undersampling
    model.fit(training_features,training_labels)
        
    try:
        predicted = np.array(model.predict_proba(remaining_features))

    except: # in the case that there's no undersampling
        predicted = None
        pass
    
    # get feature importances
    sorted_feature_importance_indices = np.argsort(model.feature_importances_)
    sorted_feature_importances = np.array(model.feature_importances_)[sorted_feature_importance_indices]
    sorted_feature_names = np.array(training_feature_names)[sorted_feature_importance_indices]

    ranked_features = np.vstack([sorted_feature_importances,sorted_feature_names]).T
    
    
    for i in range(len(ranked_features)):
        aggregated_feature_importances[ranked_features[i,1]] += [ranked_features[i,0]]
    
    return predicted, predicted_comps, train_predicted, train_predicted_comps

# Run ML trials

In [10]:
a = time.time()

all_kfold_predictions = defaultdict(list)
n_undersample = 600

aggregated_feature_importances = defaultdict(list)

for i in range(n_trials):
    print(f'Running trial {i+1}.')
    # use these lines if predicting probabilities
    predicted, predicted_comps, training_predicted_probs, training_predicted_comps = run_ML(n_undersample,n_trees = trees,trial_number = i,training_feature_names = training_feature_names,aggregated_feature_importances = aggregated_feature_importances)
    
    try:        
        predicted_probabilities_synthesis = predicted_probs[:,0] 

        for i in range(len(predicted_probs)):
            all_kfold_predictions[predicted_comps[i]] += [predicted_probs[i]]
    
    except:
        pass
    
    try:
        for i in range(len(training_predicted_probs)):
            all_kfold_predictions[training_predicted_comps[i]] += [training_predicted_probs[i]]
    except:
        pass
b = time.time()
print(f'Finished in {b-a} seconds.')

Running trial 1.
Running trial 2.
Running trial 3.
Running trial 4.
Running trial 5.
Running trial 6.
Running trial 7.
Running trial 8.
Running trial 9.
Running trial 10.
Finished in 79.6895580291748 seconds.


# Aggregate feature importances and print

In [11]:
means = []

for key in aggregated_feature_importances.keys():
    importances = np.array(aggregated_feature_importances[key],dtype=float)
    means += [np.mean(importances)]
    
feature_info = pd.DataFrame(data={'Name':list(aggregated_feature_importances.keys()),
                                 'Importances':means}).sort_values(by='Importances',ascending=False,inplace=False)

print(f'There are {len(feature_info)} features total.')
print(feature_info.head(10))

There are 529 features total.
                                     Name  Importances
528                             Stability     0.055603
526    MagpieData maximum MendeleevNumber     0.022912
525        MagpieData mode CovalentRadius     0.022668
527     MagpieData minimum CovalentRadius     0.022665
524        MagpieData avg_dev GSvolume_pa     0.011175
522            MagpieData mean NpUnfilled     0.009652
523     MagpieData avg_dev CovalentRadius     0.009552
514       MagpieData range CovalentRadius     0.008932
521  cub_oct_MagpieData minimum NpValence     0.008816
498          MagpieData avg_dev NpValence     0.008628


# Aggregate probabilities from n trials and write results to csv file

In [12]:
results_comps = []
results_synth_probs = []
results_e_hull = []
results_experimental = []
results_e_count = []
results_band_gap = []



for comp in all_kfold_predictions:
    
    
    try:
        # record E hull

        e1 = original_data[original_data['Composition'] == comp]['EP 1 Ehull (eV/atom)'].values[0]
        e2 = original_data[original_data['Composition'] == comp]['EP 2 Ehull (eV/atom)'].values[0]
        e3 = original_data[original_data['Composition'] == comp]['EP 3 Ehull (eV/atom)'].values[0]
    
    except:
        # record E hull
        
        e1 = predict_data[predict_data['Composition'] == comp]['EP 1 Ehull (eV/atom)'].values[0]
        e2 = predict_data[predict_data['Composition'] == comp]['EP 2 Ehull (eV/atom)'].values[0]
        e3 = predict_data[predict_data['Composition'] == comp]['EP 3 Ehull (eV/atom)'].values[0]

    variant_energies = np.array([e1,e2,e3])
    ehull_index = np.argmin(variant_energies)
    ehull = variant_energies[ehull_index]
    
    
    
    try:
        exp = predict_data[predict_data['Composition'] == comp]['Experimental Structure'].values[0]
    except:
        exp = original_data[original_data['Composition'] == comp]['Experimental Structure'].values[0]
        
    results_e_hull += [ehull]    
    # record composition
    results_comps += [comp]
    # record synth probability
    comp_probs = np.array(all_kfold_predictions[comp])[:,0]
    avg_prob = np.mean(comp_probs)
    results_synth_probs += [avg_prob]
    # record experimental result
    results_experimental += [exp]
    # record electron count
    results_e_count += [electronCount(comp)]
    
    
results = pd.DataFrame({'Composition': results_comps, 'Synthesizability': results_synth_probs, 'Half Heusler (F-43m)': results_e_hull, 'Experimental Structure': results_experimental, 'Electron Count': results_e_count} )
results.sort_values(by=['Synthesizability'],ascending=False,inplace=True)
results.to_csv(output_filename)